In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Trabajo Práctico 1
Se buscó analizar si existe un sesgo en el dataset de snli donde el contenido mismo del texto puede estar delatando si existe una contradicción con respecto a la hipótesis oculta.
## Carga de Datos
Se cargaron los datasets correspondientes para observación, ya divididos en datos de entrenamiento, validación y prueba.

In [5]:
# Cargo los datos
df_train = pd.read_hdf("/kaggle/input/sesgos-en-el-dataset-de-snli/train_data.hdf5")
df_valid = pd.read_hdf("/kaggle/input/sesgos-en-el-dataset-de-snli/valid_data.hdf5")
df_test = pd.read_hdf("/kaggle/input/sesgos-en-el-dataset-de-snli/test_data.hdf5")

In [6]:
df_submission = pd.read_csv("/kaggle/input/sesgos-en-el-dataset-de-snli/submission_sample.csv", index_col="pairID")

In [40]:
text_train = df_train["text"].tolist()
labels_train = df_train["gold_label"].tolist()
text_val = df_valid["text"].tolist()
labels_val = df_valid["gold_label"].tolist()
text_test = df_test["text"].tolist()

In [7]:
#Veamos el balance de clases
from collections import Counter
Counter(labels_train)

## Pre-procesamiento de Texto
+ NLTK (Natural Language Toolkit)
  + Lemmatization: reduce a sus significados (ej, quita conjugación verbal)
  + Stop Words: quita preposiciones (como palabras muy usuales de relleno?)
  + Stemming: reduce las palabras a su raíz
  + Filtrado de no palabras

In [8]:
# Paquetes de Natural Language Tool Kit
import nltk
#Tokenización (a partir de este se trabajan las otras combinacionies)
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [10]:
# Función con la cual también decido cómo pre-procesar
def text_filter(dataset, do_lemm, do_stop, do_stem, do_alpha):
    texts_filtrados = list()
    for idx in range(len(dataset.text)):
        if idx%100==0:
            print("\r Procesados: {}".format(idx),end="")
        em = dataset.text[idx]
        tok = word_tokenize(em)
        if do_lemm == True:
            lem = [lemmatizer.lemmatize(x,pos='v') for x in tok]
        else:
            lem = tok
        if do_stop == True:
            stop = [x for x in lem if x not in stopwords.words('english')]
        else:
            stop = lem
        if do_stem == True:
            stem = [stemmer.stem(x) for x in stop]
        else:
            stem = stop
        if do_alpha == True:
            alpha = [x for x in stem if x.isalpha()]
        else:
            alpha = stem
        texts_filtrados.append(" ".join(alpha))
    return texts_filtrados

## Vectorizadores
+ Count Vectorizer
+ TFIDF Vectorizer

In [11]:
#Importo los vectorizadores
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [12]:
df_max = 0.99 # max_df: int para frecuencia contada, float para proporcional
df_min = 0.01 # min_df: idem
n_range = (1,1) # ngram_range: (1,1) default
cv_cv = CountVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)

In [13]:
df_max = 0.99
df_min = 0.01
n_range = (1,1)
cv_idf = TfidfVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)

In [14]:
def get_cvs(text_train, text_valid, cv):
    cv_train = cv.fit_transform(text_train)
    cv_valid = cv.transform(text_valid)
    return cv_train, cv_valid

## Calisificadores
+ Multinomial Naive-Bayes
+ Regresión Logística (MLP)

In [15]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
# Parámetros del Clasificador
a = 1
clf_NBMN = MultinomialNB(alpha = a)

In [ ]:
# Insertar celdas para MLP

## Métricas
+ Primaria:
 + asdf
+ Secundarias:
 + Precision
 + Recall
 + F1-score
 + ROC-AUC

In [17]:
from sklearn import metrics

In [18]:
def get_scores(clf, X_train, y_train, X_valid, y_valid):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    
    score_train = clf.score(X_train, y_train)
    score_valid = clf.score(X_valid, y_valid)
    return (score_train, score_valid)
    
def get_metrics(clf, X_valid, y_valid):
    y_pred = clf.predict(X_test)
    m_conf = metrics.confusion_matrix(y_valid, y_pred)
    precision = metrics.precision_score(y_valid, y_pred)
    recall_score = metrics.recall_score(y_valid,y_pred)
    f1_score = metrics.f1_score(y_valid,y_pred)
    acc = metrics.accuracy_score(y_valid, y_pred)
    
    return score_train, score_valid, m_conf, precision, recall_score, f1_score, acc

In [30]:
# Función que hace todo lo anterior de una
def process_data(clf, cv, df_train, df_valid, do_lemm, do_stop, do_stem, do_alpha):
    print("\n  Filtrando Textos")
    texts_train = text_filter(df_train, do_lemm, do_stop, do_stem, do_alpha)
    texts_valid = text_filter(df_valid, do_lemm, do_stop, do_stem, do_alpha)
    
    labels_train = df_train["gold_label"].tolist()
    labels_valid = df_valid["gold_label"].tolist()
    
    print("\nVectorizando")
    cv_train, cv_valid = get_cvs(texts_train, texts_valid, cv)
    
    print("Obteniendo Puntajes")
    score_train, score_valid = get_scores(clf, cv_train, labels_train, cv_valid, labels_valid)
    return (cv_train, cv_valid, score_train, score_valid)

In [31]:
test_raw = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, False, False, False)
test_lemm = process_data(clf_NBMN, cv_cv, df_train, df_valid, True, False, False, False)
test_stop = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, True, False, False)
test_stem = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, False, True, False)
test_alfa = process_data(clf_NBMN, cv_cv, df_train, df_valid, False, False, False, True)

In [34]:
print("Train/Valid:")
print("raw:  ",  test_raw[2],"; ",  test_raw[3])
print("lemm: ", test_lemm[2],"; ", test_lemm[3])
print("stop: ", test_stop[2],"; ", test_stop[3])
print("stem: ", test_stem[2],"; ", test_stem[3])
print("alfa: ", test_alfa[2],"; ", test_alfa[3])

Comparando los puntajes para cada prueba, parece ser que existe una mejora en train cuando se aplica lematización y stemming, mientras que la eliminación de stop words reduce el puntaje y la eliminación de no palabras no parece tener efectos significantes. Esto puede ser porque en caso de observar si una frase contradice a otra por el contenido de su vocabulario, las palabras consideradas como stop words sí están cargadas de significado para este análisis.

Remover las no palabras probablemente sea más conveniente cuando se analice el rango de ngramas.

Por lo tanto, para los siguientes parámetros, se continuará utilizando los datasets solo con Lematización y Stemming y remoción de no-palabras.

In [35]:
# Finalmente se trabaja con solo Lemmatization y Stemming
text_train_final = text_filter(df_train, True, False, True, False)
text_valid_final = text_filter(df_valid, True, False, True, False)

Definido el pre-procesamiento se analizó si se obtienen mejores resultados utilizando el CountVectorizer o TFIDFVectorizer.

In [45]:
cv_train, cv_valid = get_cvs(text_train_final, text_valid_final, cv_cv)
cv_scores = get_scores(clf_NBMN, cv_train, labels_train, cv_valid, labels_val)
cv_train, cv_valid = get_cvs(text_train_final, text_valid_final, cv_idf)
idf_scores = get_scores(clf_NBMN, cv_train, labels_train, cv_valid, labels_val)

print(cv_scores)
print(idf_scores)

Bajo las mismas condiciones el CountVectorizer parece obtener mejores resultados que el TFIDFVectorizer, por lo tanto se continuará trabajando con el CountVectorizer

In [12]:
# Empiezo a manipular los parámetros del Vectorizador
df_max = 0.99 # max_df: int para frecuencia contada, float para proporcional
df_min = 0.01 # min_df: idem
n_range = (1,1) # ngram_range: (1,1) default
cv_cv = CountVectorizer(max_df = df_max, min_df= df_min, ngram_range = n_range)
cv_train, cv_valid = get_cvs(text_train_final, text_valid_final, cv_cv)

asñdlfkajsdñf

In [ ]:
clf.fit(cv_train, labels_train)
labels_pred = clf.predict(cv_valid)

In [ ]:
#Veamos cómo funciona el clasificador para train
clf.score(cv_train, labels_train)

In [ ]:
# Veamos cómo funciona el clasificador para valid
clf.score(cv_valid, labels_val)

In [ ]:
text_test_final = text_filter(df_train, True, False, False, False)
cv_test = cv.transform(text_test_final)
get_metrics(clf, cv_test, labels_test)
# test_labels = clf.predict(cv_test)

# Para armar Submission
Una vez elegido el grado de pre-procesamiento, el vectorizador y el clasificador, lo aplico sobre el el test.
+ Preprocesamiento: Lemmatization, Stemming y No-Palabras
+ Vectorizador:
+ Clasificador:

In [ ]:
#Armo el submission.csv
df_test = pd.DataFrame(data=test_labels, columns=["pred_labels"],)

In [ ]:
df_test.head()

In [ ]:
df_test.index.names = ["pairID"]

In [ ]:
df_test

In [ ]:
df_test.to_csv("submission.csv")

# Conclusión/Resumen

In [ ]:
def cv_filtrado(data, df_max, df_min, n_range):
    texts_filtrados = text_filter(data, True, True, True, True) # En orden: Lemmatization, Stop Words, Stemming, Non-Words
    count_vect = CountVectorizer(max_df = df_max, min_df = df_min, ngram_range = n_range)
    X_data = count_vect.fit_transform(texts_filtrados)
    X_data.toarray()
    return X_data   

## Carga de Datos

## Pre-procesamiento

## Vectorización

## Clasificador